### simple Genai APP USING LANGCHAIN

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")


#langsmith tracking
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [3]:
## data injegion : from the website need to screb the data

from langchain_community.document_loaders import WebBaseLoader

In [4]:
loader=WebBaseLoader("https://python.langchain.com/v0.1/docs/langsmith/walkthrough/")
loader

In [5]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/langsmith/walkthrough/', 'title': 'LangSmith Walkthrough | 🦜️🔗 LangChain', 'description': 'Open In Colab', 'language': 'en'}, page_content='\n\n\n\n\nLangSmith Walkthrough | 🦜️🔗 LangChain\n\n\n\n\n\n\n\nSkip to main contentThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchGet startedIntroductionQuickstartInstallationUse casesQ&A with RAGExtracting structured outputChatbotsTool use and agentsQuery analysisQ&A over SQL + CSVMoreExpression LanguageGet startedRunnable interfacePrimitivesAdvantages of LCELStreamingAdd message history (memory)MoreEcosystem🦜🛠️ LangSmith🦜🛠️ LangSmithLangSmith Walkthrough🦜🕸️ LangGraph🦜️🏓 LangS

In [ ]:
# for hug document need to perform chunk

## load data--> docs--> divide into chunks-->text--> vectoe embeddings-->store in vector db

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [7]:
documents

[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/langsmith/walkthrough/', 'title': 'LangSmith Walkthrough | 🦜️🔗 LangChain', 'description': 'Open In Colab', 'language': 'en'}, page_content='LangSmith Walkthrough | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/langsmith/walkthrough/', 'title': 'LangSmith Walkthrough | 🦜️🔗 LangChain', 'description': 'Open In Colab', 'language': 'en'}, page_content='Skip to main contentThis is documentation for LangChain v0.1, which is no longer actively maintained. Check out the docs for the latest version here.ComponentsIntegrationsGuidesAPI ReferenceMorePeopleVersioningContributingTemplatesCookbooksTutorialsYouTubev0.1Latestv0.2v0.1🦜️🔗LangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS Docs💬SearchGet startedIntroductionQuickstartInstallationUse casesQ&A with RAGExtracting structured outputChatbotsTool use and agentsQuery analysisQ&A over SQL + CSVMoreExpress

In [8]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [9]:
from langchain_community.vectorstores import FAISS
vectorstordb=FAISS.from_documents(documents,embeddings)

In [10]:
vectorstordb

In [ ]:
# query from vector store db
query="First, configure your environment variables to tell LangChain to log traces."
result=vectorstordb.similarity_search(query)
result[0].page_content

'You can tell LangChain which project to log to by setting the LANGCHAIN_PROJECT environment variable (if this isn\'t set, runs will be logged to the default project). This will automatically create the project for you if it doesn\'t exist. You must also set the LANGCHAIN_ENDPOINT and LANGCHAIN_API_KEY environment variables.For more information on other ways to set up tracing, please reference the LangSmith documentation.NOTE: You can also use a context manager in python to log traces usingfrom langchain_core.tracers.context import tracing_v2_enabledwith tracing_v2_enabled(project_name="My Project"):    agent.run("How many people live in canada as of 2023?")API Reference:tracing_v2_enabledHowever, in this example, we will use environment variables.%pip install --upgrade --quiet  langchain langsmith langchainhub%pip install --upgrade --quiet  langchain-openai tiktoken pandas duckduckgo-searchimport osfrom uuid import uuid4unique_id = uuid4().hex[0:8]os.environ["LANGCHAIN_TRACING_V2"] ='

In [13]:
## retrival chain ,documents chain



In [14]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [15]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """

Answer the following questions based only on the provided context:
<context>

{context}

</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)

In [16]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n\nAnswer the following questions based only on the provided context:\n<context>\n\n{context}\n\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000002D8FD51AE30>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002D8FD581720>, root_client=<openai.OpenAI object at 0x000002D8FD519AB0>, root_async_client=<openai.AsyncOpenAI object at 0x000002D8FD51B130>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 's

In [20]:

from langchain_core.documents import Document

# Invoke the document chain with input and context
result = document_chain.invoke({
    "input": "First, configure your environment variables to tell LangChain to log traces.",
    "context": [
        Document(page_content="First, configure your environment variables to tell LangChain to log traces. "
                              "This is done by setting the LANGCHAIN_TRACING_V2 environment variable to true. "
                              "You can tell LangChain which project to log to by setting the LANGCHAIN_PROJECT "
                              "environment variable (if this isn't set, runs will be logged to the default project).")
    ]
})

# Print the result
print(result)


1. How do you enable trace logging in LangChain?

   To enable trace logging in LangChain, you need to set the `LANGCHAIN_TRACING_V2` environment variable to `true`.

2. How can you specify which project to log to in LangChain?

   You can specify which project to log to by setting the `LANGCHAIN_PROJECT` environment variable. If this isn't set, runs will be logged to the default project.


Retriver

In [23]:
retriver=vectorstordb.as_retriever()
from langchain.chains import create_retrieval_chain
retriever_chain=create_retrieval_chain(retriver,document_chain)

In [24]:
retriever_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002D8E0158D00>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n\nAnswer the following questions based only on the provided context:\n<context>\n\n{context}\n\n</context>\n\n\n'), additional_kwargs={})])
       

In [25]:
## get the resonse from the llm

response=retriever_chain.invoke({"input":"First, configure your environment variables to tell LangChain to log traces."})

In [26]:
response['answer']

'Based on the provided context, here are the answers to your potential questions:\n\n1. **How do you set up LangChain to log to a specific project?**\n   - You can set LangChain to log to a specific project by setting the `LANGCHAIN_PROJECT` environment variable. If this variable is not set, logs will default to the project that is automatically created. Additionally, you need to set `LANGCHAIN_ENDPOINT` and `LANGCHAIN_API_KEY` environment variables.\n\n2. **How can you log traces using a context manager in Python?**\n   - You can log traces using a context manager in Python with the following code snippet:\n     ```python\n     from langchain_core.tracers.context import tracing_v2_enabled\n     with tracing_v2_enabled(project_name="My Project"):\n         agent.run("How many people live in Canada as of 2023?")\n     ```\n\n3. **What is required to start using LangSmith?**\n   - To start using LangSmith, you need to create a LangSmith account and an API key. LangSmith is in closed beta

In [27]:
response['context']

[Document(metadata={'source': 'https://python.langchain.com/v0.1/docs/langsmith/walkthrough/', 'title': 'LangSmith Walkthrough | 🦜️🔗 LangChain', 'description': 'Open In Colab', 'language': 'en'}, page_content='You can tell LangChain which project to log to by setting the LANGCHAIN_PROJECT environment variable (if this isn\'t set, runs will be logged to the default project). This will automatically create the project for you if it doesn\'t exist. You must also set the LANGCHAIN_ENDPOINT and LANGCHAIN_API_KEY environment variables.For more information on other ways to set up tracing, please reference the LangSmith documentation.NOTE: You can also use a context manager in python to log traces usingfrom langchain_core.tracers.context import tracing_v2_enabledwith tracing_v2_enabled(project_name="My Project"):    agent.run("How many people live in canada as of 2023?")API Reference:tracing_v2_enabledHowever, in this example, we will use environment variables.%pip install --upgrade --quiet  l